In [2]:
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd
import json

In [3]:
loc = 'India'

with open(f'docs/pars/pars_nods_{loc}.json', 'r') as f:
    post = json.load(f)


In [4]:
class I:
    NonTB = 0
    Asym = 1
    Sym = 2
    ExCS = 3
    TxPub = 4
    TxEng = 5
    TxPri = 6
    FpPub = 7
    FpEng = 8
    FpPri = 9

In [5]:
post[0]

{'prv0': 0.0024,
 'adr': 0.0454,
 'p_pub': 0.6683,
 'p_ppm': 0.8196,
 'r_sym': 1.0114,
 'r_aware': 2.6881,
 'r_acf': 0.0038,
 'r_det_all': 3.0331,
 'r_sc': 0.2673,
 'txi_pri': 0.027,
 'ppv_pri': 0.3388,
 'dur_pri': 0.8233,
 'p_pri_on_pub': 0.8437,
 'ra': 0.2699,
 'rs': 0.3899,
 'rc': 0.3899,
 'r_det': 3.4819,
 'a0': 2.9983,
 'c0': 0.7958,
 'pr_a': 0.6254,
 'pr_s': 0.2086,
 'pr_c': 0.166,
 'prv_a': 0.0015,
 'prv_s': 0.0005,
 'prv_c': 0.0004,
 'prv_t_eu': 0.0006,
 'prv_t_ei': 0.0001,
 'prv_t_pri': 3.0244e-06,
 'det_pub': 0.6683,
 'det_eng': 0.2718,
 'det_pri': 0.0599,
 'prv_t_pub': 0.0005,
 'scale_dur': 1,
 'r_death_a': 0,
 'r_death_s': 0.12,
 'ppv_pub': 0.85,
 'ppv_eng': 0.85,
 'sens_acf': 0.7,
 'spec_acf': 0.99,
 'dur_pub': 0.5,
 'p_txd_pub': 0.0416,
 'p_txs_pub': 0.8245,
 'p_txl_pub': 0.1339,
 'p_txd_eng': 0.0252,
 'p_txs_eng': 0.6908,
 'p_txl_eng': 0.284,
 'p_txi_pub': 0.9416,
 'p_txi_eng': 0.8841}

In [11]:
class Model:
    def get_y0(self, t, pars):
        k = np.exp(- pars['adr'] * (t - 2020))
        prev = k * pars['prv0']

        p_det = np.array([pars['p_pub'], (1 - pars['p_pub']) * pars['p_ppm'], (1 - pars['p_pub']) * (1 - pars['p_ppm'])])

        p_txi = np.array([pars['p_txi_pub'], pars['p_txi_eng'], pars['txi_pri']])
        
        r_det_all = pars['r_det_all'] #(pars['r_det'] * p_entry * p_txi).sum()
        r_det_all = pars['r_det'] * (p_det * p_txi).sum()
        r_acf = pars['r_acf'] * pars['sens_acf']
        rs = rc = pars['r_sc'] + pars['r_death_s'] + r_acf

        asc = np.array([
            (rs + pars['r_aware'] - pars['adr']) / pars['r_sym'],
            1,
            pars['r_aware'] / (rc + r_det_all - pars['adr'])
        ])
        asc /= asc.sum()

        y0 = np.zeros(10)
        y0[I.NonTB] = 1 - prev
        y0[I.Asym] = asc[0] * prev
        y0[I.Sym] = asc[1] * prev
        y0[I.ExCS] = asc[2] * prev
        return y0
    
    def calc(self, t, y, pars):
        calc = dict()

        calc['onset'] = onset = pars['r_sym'] * y[I.Asym]
        calc['aware'] = pars['r_aware'] * y[I.Sym]
    
        p_det = np.array([pars['p_pub'], (1 - pars['p_pub']) * pars['p_ppm'], (1 - pars['p_pub']) * (1 - pars['p_ppm'])])
        p_txi = np.array([pars['p_txi_pub'], pars['p_txi_eng'], pars['txi_pri']])
        r_det_all = pars['r_det_all']
        r_det_all = pars['r_det'] * (p_det * p_txi).sum()
        calc['det'] = r_det_all * y[I.ExCS]

        calc['sc_a'] = sc_a = pars['r_sc'] * y[I.Asym]
        calc['sc_s'] = pars['r_sc'] * y[I.Sym]
        calc['sc_c'] = pars['r_sc'] * y[I.ExCS]
        calc['die_a'] = die_a = pars['r_death_a'] * y[I.Asym]
        calc['die_s'] = pars['r_death_s'] * y[I.Sym]
        calc['die_c'] = pars['r_death_s'] * y[I.ExCS]

        calc['acf_a'] = acf_a = pars['r_acf'] * pars['sens_acf'] * y[I.Asym]
        calc['acf_s'] = pars['r_acf'] * pars['sens_acf'] * y[I.Sym]
        calc['acf_c'] = pars['r_acf'] * pars['sens_acf'] * y[I.ExCS]

    
        calc['inc'] = onset + acf_a + sc_a + die_a - pars['adr'] * y[I.Asym]

        return calc

    def __call__(self, t, y, pars):
        calc = self.calc(t, y, pars)
        dy = np.zeros_like(y)
        
        inc, onset, aware = calc['inc'], calc['onset'], calc['aware']
        sc_a, sc_s, sc_c = calc['sc_a'], calc['sc_s'], calc['sc_c']
        die_a, die_s, die_c = calc['die_a'], calc['die_s'], calc['die_c']
        acf_a, acf_s, acf_c = calc['acf_a'], calc['acf_s'], calc['acf_c']
        det = calc['det']
        
        dy[I.Asym] += inc - onset - acf_a - sc_a - die_a
        dy[I.Sym] += onset - aware - acf_s - sc_s - die_s
        dy[I.ExCS] += aware - det - acf_c - sc_c - die_c
        return dy
    
    def measure(self, t, y, pars):
        calc = self.calc(t, y, pars)
        mea = {'Time': t}
        mea['N'] = n = y.sum()
        mea['PrevA'] = y[I.Asym]
        mea['PrevS'] = y[I.Sym]
        mea['PrevC'] = y[I.ExCS]
        mea['Inc'] = calc['inc']
        mea['DetR'] = calc['det'].sum()
        return mea
    
    
    def simulate(self, pars):
        y0 = self.get_y0(2015, pars)
        ys = solve_ivp(self, [2015, 2025], y0 = y0, args = (pars, ), dense_output=True)
        ms = [self.measure(t, ys.sol(t), pars) for t in np.linspace(2015, 2025, 21)]
        ms = pd.DataFrame(ms)
        return ys, ms
        
    

In [12]:
m = Model()
ys, ms = m.simulate(post[0])

In [14]:
- np.diff(np.log(ms.PrevC)) * 2, post[0]['adr']

(array([0.0454    , 0.04540001, 0.0454    , 0.04539999, 0.04539999,
        0.04540014, 0.04540062, 0.04540085, 0.04540086, 0.04540072,
        0.04540047, 0.04540017, 0.04539987, 0.04539959, 0.04539938,
        0.04539924, 0.04539918, 0.04539921, 0.04539929, 0.04539941]),
 0.0454)

In [77]:
ys = solve_ivp(fn, [2015, 2025], y0 = y0, args = (p, ))

In [93]:
m.calc(2015, m.get_y0(2015, pars), pars)

{'onset': 0.0023904670944685223,
 'aware': 0.0021189126810792175,
 'det': array([1.37435449e-03, 5.24939999e-04, 3.52863407e-06]),
 'sc_a': 0.0006317696799994423,
 'sc_s': 0.00021070100057753613,
 'sc_c': 0.0001676654267631957,
 'die_a': 0.0,
 'die_s': 9.4590797116739e-05,
 'die_c': 7.527067419223152e-05,
 'acf_a': 6.286971001865008e-06,
 'acf_s': 2.096762669421048e-06,
 'acf_c': 1.6684999445944653e-06,
 'inc': 0.002921219804310179}